In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# unzip
import zipfile, os, shutil

dataset = '/content/gdrive/My Drive/Colab_Notebooks/DL/datasets/cifar10.zip'
dst_path = '/content/cifa10'
dst_file = os.path.join(dst_path, 'cifar10.zip')

if not os.path.exists(dst_path):
  os.makedirs(dst_path)

# copy zip file
shutil.copy(dataset, dst_file)
  
with zipfile.ZipFile(dst_file, 'r') as file:
  file.extractall(dst_path)

train_dir = os.path.join(dst_path, 'cifar10/train')

test_dir = os.path.join(dst_path, 'cifar10/test')

print('total training  images:', len(os.listdir(train_dir)))

print('total test images:', len(os.listdir(test_dir)))

Mounted at /content/gdrive
total training  images: 10
total test images: 10


In [6]:
# path
train_path = "/content/cifa10/cifar10/train/"
test_path = "/content/cifa10/cifar10/test/"

# library
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import scipy



# load Neural Network Model Library => condition 3 of assignment
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
# Set generator with rescaler(1./255) -> condition 2 of assignment
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)

# make a loading data flow from path. It generates data at each batch sizes -> condition 1 of assignment
batch_size_num = 100
train_generator = train_datagen.flow_from_directory(train_path,
                                                    classes=sorted(os.listdir(train_path)),
                                                    batch_size = batch_size_num,
                                                    target_size = (256, 256),
                                                    subset="training",
                                                    class_mode='categorical')

valid_generator = train_datagen.flow_from_directory(train_path, 
                                                    classes=sorted(os.listdir(train_path)), 
                                                    batch_size = batch_size_num, 
                                                    target_size=(256, 256), 
                                                    subset="validation",
                                                    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(test_path, 
                                                  classes=sorted(os.listdir(test_path)),
                                                  batch_size = 100,
                                                  target_size = (256, 256),
                                                  class_mode='categorical')

print()
# check shape of data shape
print("check shape of data shape")
for x_data, class_data in train_generator:
    print(f"input data shape from train_generator: {x_data.shape}")
    print(f"class data shape from train_generator: {class_data.shape}")
    break

for x_data, class_data in valid_generator:
    print(f"input data shape from valid_generator: {x_data.shape}")
    print(f"class data shape from valid_generator: {class_data.shape}")
    break
    
for x_data, class_data in test_generator:
    print(f"input data shape from test_generator: {x_data.shape}")
    print(f"class data shape from test_generator: {class_data.shape}")
    break    

Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.

check shape of data shape
input data shape from train_generator: (100, 256, 256, 3)
class data shape from train_generator: (100, 10)
input data shape from valid_generator: (100, 256, 256, 3)
class data shape from valid_generator: (100, 10)
input data shape from test_generator: (100, 256, 256, 3)
class data shape from test_generator: (100, 10)


In [8]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

def create_alexnet_model():
# 모델 구성: AlexNet 모델
  model = Sequential()

  # Convolutional layers  
  model.add(Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu', input_shape=(256, 256, 3)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

  model.add(Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

  model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'))

  model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'))

  model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))

  model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
  model.add(Flatten())

  # Fully-connected layers
  model.add(Dense(4096, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Dense(4096, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Dense(1000, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Dense(10, activation='softmax'))

  return model


In [10]:
def get_adaptation_batch(instance_x, instance_y, batch_size = 32):
    indices = np.random.choice(len(instance_x), size=batch_size, replace=False)
    batch_x = instance_x[indices]
    batch_y = instance_y[indices]
    return batch_x, batch_y

In [ ]:
# MAML 훈련
num_epochs = 10  # 적절한 훈련 반복 횟수 설정
num_tasks = 5  # 훈련 데이터로부터 생성할 작업 수

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss_epoch = 0.0
    train_acc_epoch = 0.0

    for task in range(num_tasks):
        # 작업 데이터로부터 인스턴스 생성
        instance_x, instance_y = train_generator.next()  # 적절한 작업 데이터에서 인스턴스 생성 함수를 사용하여 인스턴스 데이터 생성
        
        # 새로운 모델 생성 및 초기 파라미터 설정
        model = create_alexnet_model()  # 적절한 AlexNet 모델 생성 함수를 사용하여 모델 생성
        model.set_weights(initial_weights)  # 초기 파라미터 설정 -> 사전 훈련된 모델의 가중치로 설정
        
        # 작업 적응 단계 진행 -> task임 num_adaptation_step이 5면은 task가 5개라는 소리임 
        for adaptation_step in range(num_adaptation_steps): # 보통 1~5의 값을 넣음 근데 초기에는 1~3정도를 넣어보고 실험을 해보면서 모델성능이 좋아지는봐야함 숫자 클수록 좋지만 그만큼 학습속도도 오래걸리고 오버피팅이 걸림
            # 작업 데이터에서 배치 생성
            batch_x, batch_y = get_adaptation_batch(instance_x, instance_y)  # 적절한 작업 데이터에서 배치 생성 함수를 사용하여 배치 생성
            
            # 모델 업데이트 (적응 단계에서 역전파 진행)
            model.train_on_batch(batch_x, batch_y)  # 모델의 파라미터 업데이트

        # 훈련 데이터로 평가
        train_loss, train_acc = model.evaluate(instance_x, instance_y)
        train_loss_epoch += train_loss
        train_acc_epoch += train_acc
        
        # 검증 데이터로 평가
        val_loss, val_acc = model.evaluate(valid_generator)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

     # 평균 훈련 손실과 정확도 계산
    train_loss_epoch /= num_tasks
    train_acc_epoch /= num_tasks
    train_losses.append(train_loss_epoch)
    train_accuracies.append(train_acc_epoch)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss_epoch:.4f} - Train Accuracy: {train_acc_epoch:.4f}")
    print(f"Val Loss: {val_loss:.4f} - Val Accuracy: {val_acc:.4f}")
    print()


In [ ]:
# 테스트 데이터로 평가
test_loss, test_acc = model.evaluate(test_generator)  
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)